In [1]:
import os

from iexfinance import refdata
import pandas as pd

In [2]:
SANDBOX = True

if SANDBOX:
    os.environ['IEX_TOKEN'] = 'XXXXXXXXX'
    os.environ['IEX_API_VERSION'] = 'iexcloud-sandbox'
else:
    os.environ['IEX_TOKEN'] = 'XXXXXXXXX'

In [3]:
ticker_map = pd.read_csv('https://numerai-signals-public-data.s3-us-west-2.amazonaws.com/signals_ticker_map_w_bbg.csv')
ticker_map[['symbol', 'region']] = ticker_map['bloomberg_ticker'].str.split(' ', expand=True)

iex = []
# TODO: doing this for every region needs more work,
# not every region maps perfectly
for region in ['US']: #ticker_map['region'].unique():
    iex.append(refdata.get_region_symbols(region=region))
iex = pd.concat(iex)

In [4]:
# join old map with iex dataset based on split bloomberg ticker
extended_map = pd.merge(ticker_map, iex, how='left', on='symbol', suffixes=(None, '_iex'))[[
    'ticker',
    'bloomberg_ticker',
    'yahoo',
    'symbol',
    'region',
    'exchange',
    'iexId',
    'figi', # https://www.openfigi.com/about/symbology
    'cik', # https://www.sec.gov/edgar/searchedgar/cik.htm
    'lei', # https://en.wikipedia.org/wiki/Legal_Entity_Identifier
]]
extended_map = extended_map.rename({'iexId': 'iex_id'}, axis=1)

In [5]:
# TODO: dropping all other regions for now
extended_map_us = extended_map[extended_map['region'] == 'US']

In [6]:
# couple of weird outliers that could not be joined
extended_map_us[(extended_map_us['iex_id'].isna()) & (extended_map_us['figi'].isna())]

,ticker,bloomberg_ticker,yahoo,symbol,region,exchange,iex_id,figi,cik,lei
1780,BF.A,BF/A US,BF-A,BF/A,US,NaN,NaN,NaN,NaN,NaN
1781,BF.B,BF/B US,BF-B,BF/B,US,NaN,NaN,NaN,NaN,NaN
1904,BRK.B,BRK/B US,BRK-B,BRK/B,US,NaN,NaN,NaN,NaN,NaN
2307,CWEN.A,CWEN/A US,CWEN-A,CWEN/A,US,NaN,NaN,NaN,NaN,NaN
2834,GEF.B,GEF/B US,GEF-B,GEF/B,US,NaN,NaN,NaN,NaN,NaN
3435,LEN.B,LEN/B US,LEN-B,LEN/B,US,NaN,NaN,NaN,NaN,NaN
3440,LGF.A,LGF/A US,LGF-A,LGF/A,US,NaN,NaN,NaN,NaN,NaN
3441,LGF.B,LGF/B US,LGF-B,LGF/B,US,NaN,NaN,NaN,NaN,NaN
3690,MOG.A,MOG/A US,MOG-A,MOG/A,US,NaN,NaN,NaN,NaN,NaN


In [7]:
extended_map_us

,ticker,bloomberg_ticker,yahoo,symbol,region,exchange,iex_id,figi,cik,lei
1266,A,A US,A,A,US,NYS,IEX_46574843354B2D52,62BB0V3C0D0G,1129321,9WM7QAYX282I7PXUWG0R
1269,AA,AA US,AA,AA,US,NYS,IEX_4238333734532D52,0TB3H30DG3BB,1708541,1P3640UW21TW9EF0952Z
1273,AAL,AAL US,AAL,AAL,US,NAS,IEX_4353464A535A2D52,G58087BQPB01,6297,B31WIU6O4D4WX6B6QXDT
1277,AAOI,AAOI US,AAOI,AAOI,US,NAS,IEX_4C394B5933512D52,B1V06D50W0BG,1189610,55TQ09D68YUI9N0S4L82
1278,AAP,AAP US,AAP,AAP,US,NYS,IEX_4850353858392D52,BFC01JG00B7R,1171131,95F0YH9U00Q95SNC270L
...,...,...,...,...,...,...,...,...,...,...
5426,ZS,ZS US,ZS,ZS,US,NAS,IEX_5654483838362D52,43H30B3083GB,1744943,9M20B0KFNRZ619WRT450
5427,ZTS,ZTS US,ZTS,ZTS,US,NYS,IEX_5457364B4B562D52,B0N09B3G2930,1568783,J5Q9K3H4DL0891L049OC
5428,ZUMZ,ZUMZ US,ZUMZ,ZUMZ,US,NAS,IEX_4D4352464E582D52,0BPG8XY100B2,1383029,536PP5JD8H4Z0D0F5290
5429,ZUO,ZUO US,ZUO,ZUO,US,NYS,IEX_4D474631525A2D52,5HBBGGT0B300,1438979,V34QCIJ00G02541SZWY9


In [8]:
extended_map_us.to_parquet('extended_map_us.parquet')